In [1]:
import os
import shutil
import glob
import re
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple, Optional, Union
import warnings
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from tqdm import tqdm
import logging

# Setup logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [21]:
CONFIG = {
    "folders": "E:/wd_camera_test/2025_01_WCAM_originals/outputs",
    "detection_training_dir": "E:/wd_camera_test/2025_01_WCAM_originals/training/detection_training",
    "classification_training_dir": "E:/wd_camera_test/2025_01_WCAM_originals/training/classification_training",
    "ai_classification_csv": "C:/Users/willo/OneDrive - UNSW/Documents/Work/CES/Wild Deserts/Image classification/coding/july_2025_pipeline/config_2025-07-28.csv",
    "parent_dir": "E:/wd_camera_test/",
    "remote_dir": "/home/willwright/Documents/will_drive/wild_deserts_outputs/",
    "image_width": 2048,
    "image_height": 1440,
    "timezone": "Australia/Sydney",
    "species_classes": [
        "Kangaroo",
        "Cat",
        "Rabbit",
        "Dingo",
        "Fox",
        "Bilby",
        "Quoll",
        "Unidentifiable",
        "Bettong",
        "Crest-tailed mulgara",
        "Dusky hopping mouse",
        "Golden bandicoot",
        "Greater bilby",
        "Western quoll",
    ],
    "species_regex": r"Bilby|blobs|Cat|Dingo|Fox|Kangaroo|Quoll|non_target|Rabbit",
    "threshold_for_timeblocks": 600,  # 10 minutes in seconds
    "output_csv_name": f"processed_camera_trap_data_{datetime.now().strftime('%Y-%m-%d')}.csv",
}


In [5]:

def extract_image_metadata(
    images: List[str], tags: List[str] = ["Categories"]
) -> pd.DataFrame:
    """
    Extract metadata from images using exiftool (required for EXIF Categories).

    Args:
        images: List of image file paths
        tags: EXIF tags to extract (default: ["Categories"])

    Returns:
        DataFrame with image metadata
    """
    logger.info(f"Processing {len(images)} images...")

    start_time = time.time()

    try:
        from exiftool import ExifToolHelper

        metadata_list = []

        # Process images in batches for better performance
        batch_size = 100
        total_batches = (len(images) + batch_size - 1) // batch_size

        with ExifToolHelper() as et:
            for batch_idx in range(total_batches):
                start_idx = batch_idx * batch_size
                end_idx = min(start_idx + batch_size, len(images))
                batch_images = images[start_idx:end_idx]

                logger.info(
                    f"Processing batch {batch_idx + 1}/{total_batches} ({len(batch_images)} images)"
                )

                try:
                    # Extract metadata for entire batch
                    batch_metadata = et.get_metadata(batch_images)

                    for img_path, metadata in zip(batch_images, batch_metadata):
                        # Look for Categories in multiple possible EXIF fields
                        categories = (
                            metadata.get("XMP:Categories", "")
                            or metadata.get("IPTC:Keywords", "")
                            or metadata.get("XMP:Subject", "")
                            or metadata.get("EXIF:Categories", "")
                            or metadata.get("XMP:Categories", "")
                            or ""
                        )

                        # If categories is a list, join it
                        if isinstance(categories, list):
                            categories = ", ".join(categories)

                        metadata_list.append(
                            {"SourceFile": img_path, "Categories": str(categories)}
                        )

                except Exception as e:
                    logger.warning(f"Failed to extract metadata from batch: {e}")
                    # Add empty entries for failed batch
                    for img_path in batch_images:
                        metadata_list.append({"SourceFile": img_path, "Categories": ""})

        df = pd.DataFrame(metadata_list)

    except ImportError:
        logger.error("ExifTool is required for extracting EXIF Categories tags!")
        logger.error("Install with: pip install pyexiftool")
        logger.error("And ensure exiftool binary is installed on your system")
        raise ImportError(
            "ExifTool is required for this pipeline - EXIF tags contain essential metadata"
        )

    elapsed_time = time.time() - start_time
    logger.info(f"Metadata extraction completed in {elapsed_time:.2f} seconds")
    logger.info(f"Extracted Categories from {len(df[df['Categories'] != ''])} images")

    return df


def process_metadata(data_df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean and process image metadata.

    Args:
        data_df: Raw metadata DataFrame

    Returns:
        Processed DataFrame with additional columns
    """
    df = data_df.copy()

    # Clean Categories column (remove "Categor" prefix like in R code)
    df["Categories"] = df["Categories"].str.replace("Categor", "", regex=False)

    # Extract correct/incorrect status from Categories tag
    df["correct_incorrect"] = df["Categories"].str.extract(
        r"(correct|incorrect)", expand=False
    )

    # Extract predicted species from file path (this comes from filename/path structure)
    df["predicted_species"] = df["SourceFile"].str.extract(
        f"({CONFIG['species_regex']})", expand=False
    )

    # Extract correct species from Categories tag (this is the key difference!)
    def get_correct_species(row):
        categories = str(row["Categories"]) if pd.notna(row["Categories"]) else ""

        if row["correct_incorrect"] == "incorrect":
            # For incorrect predictions, the correct species is in the Categories tag
            match = re.search(CONFIG["species_regex"], categories)
            return match.group(0) if match else None
        elif row["correct_incorrect"] == "correct" and "discard" not in str(
            row["SourceFile"]
        ):
            # For correct predictions, the predicted species IS the correct species
            return row["predicted_species"]
        elif "further_verification" in str(row["SourceFile"]):
            # For further verification, check Categories tag
            match = re.search(CONFIG["species_regex"], categories)
            return match.group(0) if match else None
        elif "discard" in str(row["SourceFile"]):
            return "empty"
        else:
            return None

    df["correct_species"] = df.apply(get_correct_species, axis=1)

    # Additional processing columns
    df["discard"] = df["SourceFile"].str.contains("discard", na=False)
    df["confidence"] = df["SourceFile"].apply(
        lambda x: "low_confidence" if "low_confidence" in str(x) else "high_confidence"
    )

    # Extract camera ID
    df["camera"] = df["SourceFile"].str.extract(r"([WP]CAM\d{2})", expand=False)

    # Extract and parse datetime
    datetime_pattern = r"(\d{4}_\d{2}_\d{2}_\d{2}_\d{2}_\d{2})"
    df["datetime_str"] = df["SourceFile"].str.extract(datetime_pattern, expand=False)
    df["datetime"] = pd.to_datetime(
        df["datetime_str"], format="%Y_%m_%d_%H_%M_%S", errors="coerce"
    )

    # Create ID column
    df["id"] = df["camera"].astype(str) + "_" + df["datetime"].astype(str)

    # Convert to categorical
    df["correct_species"] = pd.Categorical(df["correct_species"])
    df["predicted_species"] = pd.Categorical(df["predicted_species"])

    # Log some statistics for verification
    logger.info(f"Processed {len(df)} images:")
    logger.info(f"  - Correct predictions: {sum(df['correct_incorrect'] == 'correct')}")
    logger.info(
        f"  - Incorrect predictions: {sum(df['correct_incorrect'] == 'incorrect')}"
    )
    logger.info(f"  - Species found: {df['correct_species'].value_counts().to_dict()}")

    return df


def convert_xyxy_to_yolo(
    bbox_string: str,
    image_width: int = CONFIG["image_width"],
    image_height: int = CONFIG["image_height"],
    round_digits: int = 4,
) -> Optional[str]:
    """
    Convert bounding box coordinates from xyxy to YOLO format.

    Args:
        bbox_string: String representation of bounding box coordinates
        image_width: Image width in pixels
        image_height: Image height in pixels
        round_digits: Number of decimal places to round to

    Returns:
        YOLO format coordinates as string or None if invalid
    """
    if pd.isna(bbox_string) or bbox_string == "":
        return None

    try:
        # Clean and parse coordinates
        coords_str = re.sub(r"[\[\]]", "", str(bbox_string))
        coords = [float(x.strip()) for x in coords_str.split(",")]

        if len(coords) != 4:
            return None

        x_min, y_min, x_max, y_max = coords

        # Calculate YOLO format coordinates
        box_width = x_max - x_min
        box_height = y_max - y_min
        x_center_norm = (x_min + box_width / 2) / image_width
        y_center_norm = (y_min + box_height / 2) / image_height
        width_norm = box_width / image_width
        height_norm = box_height / image_height

        return " ".join(
            [
                str(round(x_center_norm, round_digits)),
                str(round(y_center_norm, round_digits)),
                str(round(width_norm, round_digits)),
                str(round(height_norm, round_digits)),
            ]
        )

    except Exception as e:
        logger.warning(f"Failed to convert bbox: {bbox_string} - {e}")
        return None


def scale_md_centre(
    bbox_string: str,
    width: int = CONFIG["image_width"],
    height: int = CONFIG["image_height"],
    round_digits: int = 3,
) -> Optional[str]:
    """
    Scale MegaDetector center coordinates.

    Args:
        bbox_string: String representation of bounding box coordinates
        width: Image width
        height: Image height
        round_digits: Number of decimal places to round to

    Returns:
        Scaled coordinates as string or None if invalid
    """
    if pd.isna(bbox_string) or bbox_string == "":
        return None

    try:
        # Clean and parse coordinates
        coords_str = re.sub(r"[\[\]]", "", str(bbox_string))
        coords = [float(x.strip()) for x in coords_str.split(",")]

        if len(coords) != 4:
            return None

        x_min, y_min, w, h = coords

        x_c = x_min + w / 2
        y_c = y_min + h / 2

        return " ".join(
            [
                str(round(x_c, round_digits)),
                str(round(y_c, round_digits)),
                str(round(w, round_digits)),
                str(round(h, round_digits)),
            ]
        )

    except Exception as e:
        logger.warning(f"Failed to scale bbox: {bbox_string} - {e}")
        return None


def safe_file_copy(
    df: pd.DataFrame, source_col: str, dest_col: str, file_type: str = "file"
) -> Dict[str, int]:
    """
    Safely copy files with error handling.

    Args:
        df: DataFrame with source and destination paths
        source_col: Column name for source paths
        dest_col: Column name for destination paths
        file_type: Type of file being copied (for logging)

    Returns:
        Dictionary with success and error counts
    """
    logger.info(f"Copying {len(df)} {file_type}s...")

    success_count = 0
    error_count = 0

    for i, row in tqdm(df.iterrows(), total=len(df), desc=f"Copying {file_type}s"):
        source = row[source_col]
        dest = row[dest_col]

        # Create destination directory
        Path(dest).parent.mkdir(parents=True, exist_ok=True)

        try:
            if Path(source).exists():
                shutil.copy2(source, dest)
                success_count += 1
                if (i + 1) % 50 == 0 or (i + 1) == len(df):
                    logger.info(f"Progress: {i + 1}/{len(df)} files copied")
            else:
                logger.warning(f"⚠ Source {file_type} not found: {source}")
                error_count += 1
        except Exception as e:
            logger.error(f"✗ Failed to copy {file_type} {Path(source).name}: {e}")
            error_count += 1

    logger.info(
        f"✓ {file_type} copy complete: {success_count} successful, {error_count} errors"
    )
    return {"success": success_count, "errors": error_count}


def write_yolo_labels(df: pd.DataFrame) -> Dict[str, int]:
    """
    Write YOLO labels to files.

    Args:
        df: DataFrame with labels and destination paths

    Returns:
        Dictionary with success and error counts
    """
    logger.info(f"Writing {len(df)} YOLO label files...")

    success_count = 0
    error_count = 0

    for i, row in tqdm(df.iterrows(), total=len(df), desc="Writing labels"):
        label = row["label"]
        dest_path = row["destination_labels"]

        try:
            Path(dest_path).parent.mkdir(parents=True, exist_ok=True)
            with open(dest_path, "w") as f:
                f.write(str(label))
            success_count += 1
            if (i + 1) % 50 == 0 or (i + 1) == len(df):
                logger.info(f"Progress: {i + 1}/{len(df)} labels written")
        except Exception as e:
            logger.error(f"✗ Failed to create label {Path(dest_path).name}: {e}")
            error_count += 1

    logger.info(
        f"✓ Label writing complete: {success_count} successful, {error_count} errors"
    )
    return {"success": success_count, "errors": error_count}


In [ ]:

def get_night_day(row):
                if "night" in str(row["SourceFile"]):
                    return "night"
                elif "day" in str(row["SourceFile"]):
                    return "day"
                elif "model" in row and row["model"] == "day":
                    return "day"
                elif "model" in row and row["model"] == "night":
                    return "night"
                else:
                    return "day"

def process_camera_trap_data() -> pd.DataFrame:
    """
    Main function to process camera trap images.

    Returns:
        Processed DataFrame with camera trap data
    """
    # 1. Load and process image metadata
    logger.info("Step 1: Loading image metadata...")

    images = glob.glob(os.path.join(CONFIG["folders"],"**" , "*.JPG"), recursive=True)

    if not images:
        raise ValueError("No images found in specified folders")

    logger.info(f"Found {len(images)} images")

    # Extract metadata
    raw_metadata = extract_image_metadata(images)

    # Process metadata
    final_df = process_metadata(raw_metadata)

    # 2. Process detection training data
    logger.info("\nStep 2: Processing detection training data...")
    process_detection_training(final_df)

    # 3. Process classification training data
    logger.info("\nStep 3: Processing classification training data...")
    process_classification_training(final_df)

    logger.info("\n✅ Pipeline completed successfully!")
    return final_df


def process_detection_training(final_df: pd.DataFrame) -> None:
    """
    Process detection training data.

    Args:
        final_df: Processed metadata DataFrame
    """
    # Process incorrect bounding boxes
    logger.info("Processing incorrect bounding boxes...")

    

    # Process all labeled detections
    logger.info("Processing all labeled detections...")

    # Load AI classification data
    if not Path(CONFIG["ai_classification_csv"]).exists():
        logger.warning(
            f"AI classification CSV not found: {CONFIG['ai_classification_csv']}"
        )
        return

    try:
        ai_classification_df = pd.read_csv(CONFIG["ai_classification_csv"])
        ai_classification_df["filename"] = ai_classification_df["blank_path"].apply(
            lambda x: Path(x).name
        )
        ai_classification_df["datetime"] = pd.to_datetime(ai_classification_df["time"])
        ai_classification_df["id"] = (
            ai_classification_df["camera"].astype(str)
            + "_"
            + ai_classification_df["datetime"].astype(str)
        )
        incorrect_bounding = final_df[
        (final_df["correct_incorrect"] == "incorrect")
        & (final_df["correct_species"] == "non_target")
        ].copy()
        if len(incorrect_bounding) > 0:

            incorrect_bounding = incorrect_bounding.merge(
            ai_classification_df, on="id", how="left"
            )
            incorrect_bounding = incorrect_bounding[
                incorrect_bounding["correct_species"] == "non_target"
            ].copy()
            incorrect_bounding["blank_path"] = incorrect_bounding[
                "blank_path"
            ].str.replace(CONFIG["remote_dir"], CONFIG["parent_dir"], regex=False)
            incorrect_bounding["detection_image_path"] = incorrect_bounding[
                "blank_path"
            ]
            incorrect_bounding["detection_label_path"] = (
                incorrect_bounding["detection_image_path"]
                .str.replace("images", "labels", regex=False)
                .str.replace(".jpg", ".txt", regex=False)
            )
            incorrect_bounding["night_day"] = incorrect_bounding.apply(
                get_night_day, axis=1
            )
            incorrect_bounding["destination_images"] = (
                CONFIG["detection_training_dir"]
                + "/"
                + incorrect_bounding["night_day"]
                + "/images/"
                + incorrect_bounding["detection_image_path"].apply(lambda x: Path(x).name)
            )
            incorrect_bounding["destination_labels"] = (
                incorrect_bounding["destination_images"]
                .str.replace("images", "labels", regex=False)
                .str.replace(".jpg", ".txt", regex=False)
            
            )

            results = safe_file_copy(
                incorrect_bounding,
                "detection_image_path",
                "destination_images",
                "detection image",
            )
            logger.info(
                f"Processed {results['success']} incorrect bounding images ({results['errors']} errors)"
            )       
        # Join with final_df
        all_labelled_detections = final_df.merge(
            ai_classification_df, on="id", how="left"
        )
        all_labelled_detections = all_labelled_detections[
            all_labelled_detections["correct_species"] != "non_target"
        ].copy()

        if len(all_labelled_detections) > 0:
            # Process paths and labels
            all_labelled_detections["blank_path"] = all_labelled_detections[
                "blank_path"
            ].str.replace(CONFIG["remote_dir"], CONFIG["parent_dir"], regex=False)
            all_labelled_detections["detection_image_path"] = all_labelled_detections[
                "blank_path"
            ]

            # Determine night/day
            

            all_labelled_detections["night_day"] = all_labelled_detections.apply(
                get_night_day, axis=1
            )

            all_labelled_detections["destination_images"] = (
                CONFIG["detection_training_dir"]
                + "/"
                + all_labelled_detections["night_day"]
                + "/images/"
                + all_labelled_detections["detection_image_path"].apply(
                    lambda x: Path(x).name if pd.notna(x) else ""
                )
            )
            all_labelled_detections["destination_labels"] = (
                all_labelled_detections["destination_images"]
                .str.replace("images", "labels", regex=False)
                .str.replace(".JPG", ".txt", regex=False)
            )

            # Create labels
            def create_label(row):
                if row["correct_species"] == "empty":
                    return None

                try:
                    species_idx = CONFIG["species_classes"].index(
                        row["correct_species"]
                    )
                except (ValueError, TypeError):
                    return None

                if "species" in row and row["species"] == "animal":
                    bbox_coords = scale_md_centre(row.get("bbox", ""))
                else:
                    bbox_coords = convert_xyxy_to_yolo(row.get("bbox", ""))

                if bbox_coords:
                    return f"{species_idx} {bbox_coords}"
                return None

            all_labelled_detections["label"] = all_labelled_detections.apply(
                create_label, axis=1
            )

            # Filter valid labels and group by ID
            valid_detections = all_labelled_detections[
                all_labelled_detections["label"].notna()
                & all_labelled_detections["detection_image_path"].notna()
            ].copy()

            if len(valid_detections) > 0:
                # Group by ID and combine labels
                grouped = (
                    valid_detections.groupby("id")
                    .agg(
                        {
                            "detection_image_path": "first",
                            "destination_images": "first",
                            "destination_labels": "first",
                            "label": lambda x: "\n".join(x.dropna().unique()),
                        }
                    )
                    .reset_index()
                )

                # Write labels
                label_results = write_yolo_labels(grouped)

                # Copy images
                image_results = safe_file_copy(
                    grouped,
                    "detection_image_path",
                    "destination_images",
                    "detection image",
                )

                logger.info(
                    f"Processed {len(grouped)} labeled detections: "
                    f"{image_results['success']} images, {label_results['success']} labels"
                )

    except Exception as e:
        logger.error(f"Error processing labeled detections: {e}")


def process_classification_training(final_df: pd.DataFrame) -> None:
    """
    Process classification training data.

    Args:
        final_df: Processed metadata DataFrame
    """
    classification_data = final_df[final_df["correct_species"] != "non_target"].copy()

    classification_data["current_image_path"] = classification_data["SourceFile"]

    def get_night_day_class(path):
        if "night" in str(path):
            return "night"
        elif "day" in str(path):
            return "day"
        else:
            return None

    classification_data["night_day"] = classification_data["current_image_path"].apply(
        get_night_day_class
    )

    classification_data["destination_images"] = (
        CONFIG["classification_training_dir"]
        + "/"
        + classification_data["night_day"]
        + "/"
        + classification_data["correct_species"].astype(str)
        + "/"
        + classification_data["correct_incorrect"].astype(str)
        + "_"
        + classification_data["current_image_path"].apply(lambda x: Path(x).name)
    )

    # Filter valid entries
    valid_data = classification_data[
        classification_data["current_image_path"].notna()
        & classification_data["destination_images"].notna()
        & classification_data["night_day"].notna()
    ].copy()

    if len(valid_data) > 0:
        results = safe_file_copy(
            valid_data,
            "current_image_path",
            "destination_images",
            "classification image",
        )

        logger.info(
            f"Processed {results['success']} classification images ({results['errors']} errors)"
        )


In [29]:
final_results = process_camera_trap_data()

2025-07-30 13:29:09,198 - INFO - Step 1: Loading image metadata...
2025-07-30 13:29:09,205 - INFO - Found 810 images
2025-07-30 13:29:09,206 - INFO - Processing 810 images...
2025-07-30 13:29:09,488 - INFO - Processing batch 1/9 (100 images)
2025-07-30 13:29:10,032 - INFO - Processing batch 2/9 (100 images)
2025-07-30 13:29:10,539 - INFO - Processing batch 3/9 (100 images)
2025-07-30 13:29:11,049 - INFO - Processing batch 4/9 (100 images)
2025-07-30 13:29:11,488 - INFO - Processing batch 5/9 (100 images)
2025-07-30 13:29:11,905 - INFO - Processing batch 6/9 (100 images)
2025-07-30 13:29:12,345 - INFO - Processing batch 7/9 (100 images)
2025-07-30 13:29:12,771 - INFO - Processing batch 8/9 (100 images)
2025-07-30 13:29:13,370 - INFO - Processing batch 9/9 (10 images)
2025-07-30 13:29:13,472 - INFO - Metadata extraction completed in 4.27 seconds
2025-07-30 13:29:13,472 - INFO - Extracted Categories from 810 images
2025-07-30 13:29:13,512 - INFO - Processed 810 images:
2025-07-30 13:29:13

In [12]:

os.path.join(CONFIG["folders"], pattern)

'E:/wd_camera_test/2025_01_WCAM_originals/outputs\\*.JPEG'

In [23]:
images = glob.glob(os.path.join(CONFIG["folders"],"**" , "*.JPG"), recursive=True)
raw_metadata = extract_image_metadata(images)

# Process metadata
final_df = process_metadata(raw_metadata)

2025-07-30 12:56:49,141 - INFO - Processing 810 images...
2025-07-30 12:56:49,541 - INFO - Processing batch 1/9 (100 images)
2025-07-30 12:56:50,168 - INFO - Processing batch 2/9 (100 images)
2025-07-30 12:56:50,804 - INFO - Processing batch 3/9 (100 images)
2025-07-30 12:56:51,674 - INFO - Processing batch 4/9 (100 images)
2025-07-30 12:56:52,322 - INFO - Processing batch 5/9 (100 images)
2025-07-30 12:56:52,927 - INFO - Processing batch 6/9 (100 images)
2025-07-30 12:56:53,581 - INFO - Processing batch 7/9 (100 images)
2025-07-30 12:56:54,191 - INFO - Processing batch 8/9 (100 images)
2025-07-30 12:56:54,954 - INFO - Processing batch 9/9 (10 images)
2025-07-30 12:56:55,095 - INFO - Metadata extraction completed in 5.95 seconds
2025-07-30 12:56:55,095 - INFO - Extracted Categories from 810 images
2025-07-30 12:56:55,157 - INFO - Processed 810 images:
2025-07-30 12:56:55,167 - INFO -   - Correct predictions: 724
2025-07-30 12:56:55,169 - INFO -   - Incorrect predictions: 12
2025-07-30 

In [24]:
incorrect_bounding = final_df[
        (final_df["correct_incorrect"] == "incorrect")
        & (final_df["correct_species"] == "non_target")
    ].copy()

if len(incorrect_bounding) > 0:
    incorrect_bounding["detection_image_path"] = incorrect_bounding["SourceFile"]
    incorrect_bounding["detection_label_path"] = (
        incorrect_bounding["detection_image_path"]
        .str.replace("images", "labels", regex=False)
        .str.replace(".jpg", ".txt", regex=False)
    )
    incorrect_bounding["night_day"] = incorrect_bounding["SourceFile"].apply(
        lambda x: "night" if "night" in str(x) else "day"
    )
    incorrect_bounding["destination_images"] = (
        CONFIG["detection_training_dir"]
        + "/"
        + incorrect_bounding["night_day"]
        + "/images/"
        + incorrect_bounding["detection_image_path"].apply(lambda x: Path(x).name)
    )
    incorrect_bounding["destination_labels"] = (
        CONFIG["detection_training_dir"]
        + "/"
        + incorrect_bounding["night_day"]
        + "/labels/"
        + incorrect_bounding["detection_label_path"].apply(lambda x: Path(x).name)
    )

    results = safe_file_copy(
        incorrect_bounding,
        "detection_image_path",
        "destination_images",
        "detection image",
    )
    logger.info(
        f"Processed {results['success']} incorrect bounding images ({results['errors']} errors)"
    )

2025-07-30 12:58:34,219 - INFO - Copying 5 detection images...
Copying detection images: 100%|██████████| 5/5 [00:00<00:00, 14.06it/s]
2025-07-30 12:58:34,574 - INFO - ✓ detection image copy complete: 5 successful, 0 errors
2025-07-30 12:58:34,575 - INFO - Processed 5 incorrect bounding images (0 errors)
